# HW7 DS610

Streaming Data

In this homework you will be running the code below to demonstrate a basic example of perform a streamed query.

There are two files attached;
* logfile1.json
* logfile2.json

Put this jupyter notebook into a local EMTPY folder on your computer. Be sure that java is installed on your computer!
* If you dont have it, download it from here; https://www.oracle.com/java/technologies/javase-downloads.html
* Check your terminal by typing "java -version" to see if you have it downloaded

Once you have this notebook in an EMPTY folder, then one at a time, add logfile1.json and logfile2.json to the folder AFTER running the script below.

In [ ]:
try:
    from pyspark.sql import SparkSession
except:
    !pip install pyspark
    from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType

import os
current_dir = os.path.abspath(os.curdir)

schema = '''
StructType([StructField('Arrival_Time', LongType(), True), 
StructField('Creation_Time', LongType(), True), StructField('Device', StringType(), True),
StructField('Index', LongType(), True), StructField('Model', StringType(), True),
StructField('User', StringType(), True), StructField('gt', StringType(), True),
StructField('x', DoubleType(), True), StructField('y', DoubleType(), True), 
StructField('z', DoubleType(), True)])
'''

spark = SparkSession.builder.config("HW7", current_dir).appName("StructuredStreaming").getOrCreate()
streaming = spark.readStream.schema(eval(schema)).json(current_dir)
# Run forever until terminated

stream = streaming.withColumn('Arrival_Time',f.to_timestamp((f.col('Arrival_Time')/1000)))\
                  .withColumn('Creation_Time',f.to_timestamp((f.col('Creation_Time')/1000000000)))\
                  .groupby('User')\
                  .agg(f.max(f.col('Arrival_Time')),f.min(f.col('Arrival_Time')))

query = stream.writeStream.outputMode("complete").format("console").queryName("arrivals").start()

query.awaitTermination()

# Cleanly shut down the session
spark.stop()